In [ ]:
from yahoo_fin import stock_info as si
from kafka import KafkaProducer
import json
import time
from sqlalchemy import create_engine
import pandas as pd

# Define as criptomoedas que deseja coletar dados
crypto_list = ["BTC-USD", "ETH-USD", "LTC-USD"]

# Define as moedas nas quais deseja obter os preços
currency_list = ["USD", "EUR", "ARS", "BRL"]

# Configurações do Apache Kafka
bootstrap_servers = ['localhost:9092']
topic_name = 'crypto_prices'

# Configurações do MySQL
db_username = 'seu_usuario'
db_password = 'sua_senha'
db_host = 'localhost'
db_name = 'crypto_data'

# Cria a conexão do SQL Alchemy
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')

# Cria o produtor do Kafka
producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                         value_serializer=lambda x: json.dumps(x).encode('utf-8'))

# Loop infinito para coletar, enviar e armazenar dados
while True:
    # Coleta os dados de preço das criptomoedas em diferentes moedas
    crypto_prices = {}
    for crypto in crypto_list:
        quote_table = si.get_quote_table(crypto)
        for currency in currency_list:
            price_key = crypto.replace("-USD", f"-{currency}")
            price_value = quote_table[f"Price ({currency})"]
            crypto_prices[price_key] = price_value

    # Envia os dados para o Kafka
    producer.send(topic_name, value=crypto_prices)

    # Cria um dataframe com os dados de preço das criptomoedas
    df = pd.DataFrame.from_dict(crypto_prices, orient='index', columns=['price'])

    # Adiciona as colunas de data e hora e de moeda
    df['timestamp'] = pd.Timestamp.now()
    df['crypto_currency'] = [crypto.split("-")[0] for crypto in df.index]
    df['fiat_currency'] = [crypto.split("-")[1] for crypto in df.index]

    # Salva os dados no banco de dados MySQL usando SQL Alchemy
    df.to_sql('crypto_prices', con=engine, if_exists='append', index=False)

    # Espera 5 segundos antes de coletar e enviar novos dados
    time.sleep(5)

